In [1]:
import pandas as pd
from functions_load_and_transform import player_list, schedule


In [9]:
def check_score(row): # check if player guessed correctly
    if row[player] == row["Winner"]:
        return 1
    else:
        return 0

In [10]:
betsDF = pd.read_csv("data/bets_2024.csv")
resultsDF = pd.read_csv("data/results.csv")
scoringDF = pd.concat([betsDF[player_list], resultsDF[["Winner", "Week", "Game Nr."]]], axis=1)

scoringDF

for player in player_list:
    scoringDF[f"score_{player}"] = scoringDF.apply(check_score, axis=1)

scoringDF


,Alex,Alina,Evelyn,Christopher,Ludwig,Manu,Natalie,Nikolai,Sebastian,Vero,...,score_Evelyn,score_Christopher,score_Ludwig,score_Manu,score_Natalie,score_Nikolai,score_Sebastian,score_Vero,score_Viki,score_Wolfgang
0,Kansas City Chiefs,NaN,Kansas City Chiefs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,Philadelphia Eagles,NaN,Philadelphia Eagles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
2,Atlanta Falcons,NaN,Atlanta Falcons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,Buffalo Bills,NaN,Buffalo Bills,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,Chicago Bears,NaN,Chicago Bears,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [3]:
betsDF = pd.read_csv("data/bets_2024.csv")
resultsDF = pd.read_csv("data/results.csv")
scoringDF = pd.read_csv("data/scoring.csv")

scoringDF["Winner"] = resultsDF["Winner"]
for player in player_list:
    scoringDF[f"bet_{player}"] = betsDF[player]
    scoringDF[player] = scoringDF.apply(check_score, axis=1)

scoringDF[["Winner", "Alex", "Evelyn", "bet_Alex", "bet_Evelyn"]].head(20)


,Winner,Alex,Evelyn,bet_Alex,bet_Evelyn
0,Kansas City Chiefs,1,1,Kansas City Chiefs,Kansas City Chiefs
1,Philadelphia Eagles,1,1,Philadelphia Eagles,Philadelphia Eagles
2,Atlanta Falcons,1,1,Atlanta Falcons,Atlanta Falcons
3,Buffalo Bills,1,1,Buffalo Bills,Buffalo Bills
4,Chicago Bears,1,1,Chicago Bears,Chicago Bears
5,Cincinnati Bengals,1,1,Cincinnati Bengals,Cincinnati Bengals
6,Indianapolis Colts,1,1,Indianapolis Colts,Indianapolis Colts
7,Miami Dolphins,1,1,Miami Dolphins,Miami Dolphins
8,New Orleans Saints,1,1,New Orleans Saints,New Orleans Saints
9,New York Giants,1,1,New York Giants,New York Giants


In [ ]:
# player DF für Zwischenstand

